In [1]:
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['figure.titlesize'] = 18
matplotlib.rcParams['figure.figsize'] = [9, 7]
matplotlib.rcParams['font.family'] = ['KaiTi']
matplotlib.rcParams['axes.unicode_minus']=False

import  os
import  tensorflow as tf
import  numpy as np
from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses

tf.random.set_seed(1234)
np.random.seed(1234)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters

In [2]:
#数据加载
(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

x = x[:10000]
y = y[:10000]

In [3]:
"""Basic case:
- We define a `build_model` function
- It returns a compiled model
- It uses hyperparameters defined on the fly
"""

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),activation='relu'))
        
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss=losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [4]:
tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,executions_per_trial=3,directory='my_dir')
print(tuner.search_space_summary())

None


In [5]:
tuner.search(x=x,y=y,epochs=3,validation_data=(val_x, val_y))
tuner.results_summary()
print("----------------------------------------------------------")

----------------------------------------------------------


In [6]:
tuner.results_summary()
print("----------------------------------------------------------")

----------------------------------------------------------


In [7]:
best_model = tuner.get_best_models(1)[0]
print("------------------------------------------------------------------------------------------")
print("best model:",best_model)
print("------------------------------------------------------------------------------------------")
print("Summary:",best_model.summary)

------------------------------------------------------------------------------------------
best model: <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001E11D14E548>
------------------------------------------------------------------------------------------
Summary: <bound method Network.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000001E11D14E548>>


In [8]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print("------------------------------------------------------------------------------------------")
print("best_hyperparameters:",best_hyperparameters)

------------------------------------------------------------------------------------------
best_hyperparameters: <kerastuner.engine.hyperparameters.HyperParameters object at 0x000001E110B6D288>


In [9]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()
model = best_model
for i,layer in enumerate(model.layers):
    if i >= 1:
        print(layer,model.layers[i].weights)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 384)               301440    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               49280     
_________________________________________________________________
dense_2 (Dense)              (None, 288)               37152     
_________________________________________________________________
dense_3 (Dense)              (None, 448)               129472    
_________________________________________________________________
dense_4 (Dense)              (None, 288)               129312    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               7

      dtype=float32)>]
<tensorflow.python.keras.layers.core.Dense object at 0x000001E11D91B288> [<tf.Variable 'dense_1/kernel:0' shape=(384, 128) dtype=float32, numpy=
array([[-0.03458741,  0.07102977, -0.08942799, ...,  0.06291483,
         0.01516281,  0.07002284],
       [ 0.06879274,  0.09559872,  0.01681753, ...,  0.09627721,
         0.03910181, -0.01786996],
       [-0.07041047,  0.09607479, -0.10742366, ..., -0.06851379,
         0.08337472,  0.13830845],
       ...,
       [-0.06032173, -0.06932592, -0.02308588, ..., -0.07766   ,
         0.09968382, -0.04611464],
       [ 0.00089261,  0.07715572,  0.07883187, ..., -0.0181727 ,
         0.11152489,  0.06992617],
       [ 0.09250982, -0.08333468,  0.08785553, ..., -0.08103116,
         0.0824496 ,  0.04148203]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(128,) dtype=float32, numpy=
array([ 0.02555429, -0.00178274, -0.00425122,  0.00106627,  0.01404636,
       -0.02650061, -0.0043186 ,  0.01553105, -0.01118614, -0.007

      dtype=float32)>]
<tensorflow.python.keras.layers.core.Dense object at 0x000001E11D152B88> [<tf.Variable 'dense_3/kernel:0' shape=(288, 448) dtype=float32, numpy=
array([[-0.02940783, -0.05752449, -0.11061397, ...,  0.04542694,
        -0.1224143 ,  0.05286772],
       [-0.0575873 , -0.02022   , -0.05895854, ..., -0.0409592 ,
        -0.01583149, -0.0538289 ],
       [-0.04711064, -0.08368696,  0.03636837, ..., -0.06695718,
        -0.01110918, -0.00266584],
       ...,
       [-0.03997738, -0.07248952,  0.0346618 , ...,  0.03786617,
         0.05225116,  0.04901042],
       [-0.07581931,  0.02209569,  0.03295497, ...,  0.01711079,
         0.02761747,  0.01652981],
       [ 0.10007445,  0.05860385, -0.0565518 , ...,  0.03690565,
         0.06715159,  0.10277043]], dtype=float32)>, <tf.Variable 'dense_3/bias:0' shape=(448,) dtype=float32, numpy=
array([ 3.31021212e-02, -1.96599141e-02, -1.05283326e-02, -3.46640460e-02,
        1.36505184e-03,  1.06666209e-02,  1.92225296e-02,  1.6

      dtype=float32)>]
<tensorflow.python.keras.layers.core.Dense object at 0x000001E11D927548> [<tf.Variable 'dense_4/kernel:0' shape=(448, 288) dtype=float32, numpy=
array([[-0.02029276, -0.09496876,  0.02729705, ..., -0.09446514,
        -0.07965299, -0.06991989],
       [ 0.01983649, -0.03431052,  0.03188546, ..., -0.02967538,
        -0.04829319, -0.00600327],
       [-0.0723314 ,  0.00712891, -0.04092664, ..., -0.02810772,
        -0.0674113 , -0.07907805],
       ...,
       [-0.02901345, -0.07147361,  0.07427247, ..., -0.0242822 ,
        -0.0334153 ,  0.1449393 ],
       [ 0.08076184, -0.04762748, -0.06047357, ..., -0.0225533 ,
        -0.00819003, -0.01857411],
       [-0.04703255,  0.00908733, -0.04594272, ..., -0.05621682,
        -0.07128271, -0.00988233]], dtype=float32)>, <tf.Variable 'dense_4/bias:0' shape=(288,) dtype=float32, numpy=
array([ 1.09721057e-03, -4.07689959e-02, -3.27375457e-02,  7.16465339e-02,
       -2.34878510e-02, -2.39168946e-02,  1.43520543e-02,  7.6

       -0.01781291], dtype=float32)>]
<tensorflow.python.keras.layers.core.Dense object at 0x000001E11D92F208> [<tf.Variable 'dense_6/kernel:0' shape=(256, 288) dtype=float32, numpy=
array([[-0.00574721,  0.01998855, -0.05607537, ...,  0.02737697,
         0.02947046,  0.12716934],
       [ 0.06026512, -0.07827175, -0.01481279, ...,  0.10012041,
         0.00642766,  0.03858585],
       [ 0.04903224,  0.09971028,  0.0488983 , ..., -0.08158401,
        -0.06049962,  0.00762568],
       ...,
       [-0.040408  , -0.07411052, -0.0694912 , ...,  0.0330983 ,
         0.03934064,  0.01921974],
       [-0.03518324,  0.03638164, -0.00759669, ...,  0.06295037,
         0.03382144, -0.07053837],
       [-0.04111535, -0.03396428,  0.00864005, ...,  0.03005506,
         0.05653501, -0.07464323]], dtype=float32)>, <tf.Variable 'dense_6/bias:0' shape=(288,) dtype=float32, numpy=
array([-0.00569759,  0.00298456,  0.00363725,  0.02914942, -0.00869756,
       -0.02734662,  0.00441982, -0.00793583,  0.0

       -0.00526703, -0.00884014,  0.01422181], dtype=float32)>]
<tensorflow.python.keras.layers.core.Dense object at 0x000001E11D93EB48> [<tf.Variable 'dense_8/kernel:0' shape=(448, 32) dtype=float32, numpy=
array([[-0.09847653,  0.00719036, -0.08139145, ..., -0.02628103,
         0.0240425 , -0.01301407],
       [-0.07951516, -0.08836254, -0.09781409, ..., -0.07947157,
        -0.06621479, -0.07280405],
       [-0.10058782,  0.01950792,  0.00644627, ...,  0.00515509,
        -0.00785935,  0.00283485],
       ...,
       [ 0.08817226, -0.05510658,  0.01158236, ..., -0.09166477,
         0.00321314,  0.10841483],
       [-0.04373014, -0.1102979 ,  0.05221585, ...,  0.01654889,
         0.10020488, -0.02035955],
       [-0.06148441,  0.04297773,  0.08146884, ...,  0.0619026 ,
        -0.12267063, -0.03685918]], dtype=float32)>, <tf.Variable 'dense_8/bias:0' shape=(32,) dtype=float32, numpy=
array([ 1.2099964e-02, -8.4696887e-03,  1.5472664e-03, -6.3045495e-03,
        2.0837002e-04,  2.2

In [10]:
model.compile(optimizer=optimizers.Adam(lr=3e-4), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
test_acc = model.evaluate(val_x, val_y,verbose=2)

print('\nTest accuracy:', test_acc)

W0908 16:46:34.947771 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0908 16:46:34.952780 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0908 16:46:34.955037 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0908 16:46:34.957035 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0908 16:46:34.959038 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0908 16:46:34.963032 16532 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


10000/10000 - 1s - loss: 1.5661 - accuracy: 0.9329

Test accuracy: [1.5660860818862916, 0.9329]


In [11]:
model = best_model
model.compile(optimizer=optimizers.Adam(lr=3e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

model.fit(x,y,epochs=10)
test_acc = model.evaluate(val_x, val_y,verbose=2)
print("test acc:", test_acc)
model.save("model.h5")
print("save model")

Train on 10000 samples
Epoch 1/10
10000/10000 [==============================] - ETA: 3:27 - loss: 1.5877 - accuracy: 0.87 - ETA: 21s - loss: 1.5375 - accuracy: 0.9563 - ETA: 10s - loss: 1.5302 - accuracy: 0.961 - ETA: 7s - loss: 1.5264 - accuracy: 0.963 - ETA: 5s - loss: 1.5228 - accuracy: 0.96 - ETA: 4s - loss: 1.5179 - accuracy: 0.96 - ETA: 3s - loss: 1.5150 - accuracy: 0.96 - ETA: 3s - loss: 1.5133 - accuracy: 0.96 - ETA: 2s - loss: 1.5120 - accuracy: 0.96 - ETA: 2s - loss: 1.5102 - accuracy: 0.96 - ETA: 2s - loss: 1.5081 - accuracy: 0.96 - ETA: 1s - loss: 1.5078 - accuracy: 0.96 - ETA: 1s - loss: 1.5066 - accuracy: 0.96 - ETA: 1s - loss: 1.5055 - accuracy: 0.96 - ETA: 1s - loss: 1.5041 - accuracy: 0.97 - ETA: 1s - loss: 1.5027 - accuracy: 0.97 - ETA: 1s - loss: 1.5021 - accuracy: 0.97 - ETA: 0s - loss: 1.5011 - accuracy: 0.97 - ETA: 0s - loss: 1.4998 - accuracy: 0.97 - ETA: 0s - loss: 1.4990 - accuracy: 0.97 - ETA: 0s - loss: 1.4977 - accuracy: 0.97 - ETA: 0s - loss: 1.4972 - accu

In [13]:
network = keras.models.load_model('model.h5')

W0908 16:46:59.563260 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0908 16:46:59.564252 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0908 16:46:59.564252 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0908 16:46:59.565248 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0908 16:46:59.566250 16532 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0908 16:46:59.567252 16532 util.py:152] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


In [14]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 384)               301440    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               49280     
_________________________________________________________________
dense_2 (Dense)              (None, 288)               37152     
_________________________________________________________________
dense_3 (Dense)              (None, 448)               129472    
_________________________________________________________________
dense_4 (Dense)              (None, 288)               129312    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               7

In [17]:
network.compile(optimizer=optimizers.Adam(lr=3e-4), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
test_acc = network.evaluate(val_x, val_y,verbose=2)

print('\nTest accuracy:', test_acc)

10000/10000 - 1s - loss: 1.5062 - accuracy: 0.9555

Test accuracy: [1.506218641281128, 0.9555]
